# Linear Regression Code Documentation

## Concept

Linear regression is a fundamental statistical method used to model the relationship between a continuous dependent variable (also called the response or target variable) and one or more independent variables (also called predictors, explanatory variables, or features). The purpose of linear regression is to establish a mathematical equation that best describes this relationship, allowing for interpretation and prediction.

### Mathematical Representation

In its simplest form, simple linear regression models the relationship between one independent variable and one dependent variable using the following equation: <dt> $$ \displaystyle Y = \beta_0 + \beta_1 X + \varepsilon $$ </dt>


    

Where:


Y → The dependent variable (response), which must be continuous

X → The independent variable (predictor), which can be continuous  or categorical

β₀  → The intercept, representing the expected value of Y when X = 0

β₁ → The regression coefficient, indicating how much Y changes for each unit increase in X.

ε → The error term, representing random variability in Y that is not explained by X.

When there are multiple independent variables, we use multiple linear regression, which extends the equation to:<dt> $$ \displaystyle Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n + \varepsilon $$ </dt>

Where:



* X₁, X₂, …, Xₙ are the independent variables.

* β₁, β₂, …, βₙ are their respective regression coefficients.

### Estimation of the Coefficients

The coefficients in linear regression are estimated using the method of Ordinary Least Squares (OLS), which minimizes the sum of the squared differences between the observed and predicted values.



### Observations


**Linearity** → The relationship between independent and dependent variables must be linear.

**Independence** → Observations must be independent of each other.

**Homoscedasticity** → The variance of residuals (errors) should be constant across all levels of X.

**Normality of Errors** → Residuals should be approximately normally distributed.

**No Multicollinearity** → Independent variables should not be highly correlated with each other.


### Interpreting Linear Regression

**1. Coefficients (β)**

In linear regression, the estimated coefficients (β) represent the change in the dependent variable for a one-unit increase in the predictor variable, assuming all other variables are held constant.

* If β > 0, an increase in the predictor variable is associated with an increase in the dependent variable.

* If β < 0, an increase in the predictor variable is associated with a decrease in the dependent variable.

* If β = 0, the predictor variable has no effect on the dependent variable.

**2. Confidence Intervals (CI)**


To assess the reliability of the estimated coefficient, we compute a confidence interval for a chosen confidence level:

The confidence interval (CI) for a regression coefficient is calculated as:

$$
\left[ \hat{\beta} - z_{\alpha/2} \cdot \mathrm{SE}(\hat{\beta}), \; \hat{\beta} + z_{\alpha/2} \cdot \mathrm{SE}(\hat{\beta}) \right]
$$
 Where:


* β̂ is the estimated regression coefficient.

* SE(β̂) is the standard error of the estimate.

* z<sub>α⁄2</sub> is the critical value from the standard normal distribution corresponding to the chosen confidence level.

If the interval does not include 0, the effect of the variable is statistically significant.

If the interval includes 0, the effect may not be significant.

**3. p-value**

The p-value tests the null hypothesis that the coefficient is zero , meaning the predictor variable has no effect on the dependent variable.

* If p < 0.05, the effect is statistically significant.

* If p ≥ 0.05, there is insufficient evidence that the variable affects the dependent variable.



### Adventages of Linear Regression

* **Simplicity**: Easy to implement and interpret.

* **Interpretability**: Coefficients have a clear and direct interpretation in terms of effect size.

* **Efficiency**: Computationally fast, especially for small to medium-sized datasets.

* **Good baseline**: Often serves as a solid baseline model before exploring more complex methods.

* **Analytical properties**: Well-understood statistical properties under standard assumptions.



### Limitations of Linear Regression


* **Linearity assumption**: Assumes a linear relationship between the predictors and the response variable, which may not hold in real-world data.

* **Sensitive to outliers**: Outliers can significantly influence the model and distort predictions.

* **Homoscedasticity required**: Assumes constant variance of the errors; violation of this can lead to inefficient estimates.

* **Normality of residuals**: Required for valid hypothesis testing and confidence intervals.

* **Multicollinearity**: High correlation between independent variables can inflate variance and make coefficients unstable.

* **Not suitable for categorical outcomes**: Cannot be used when the dependent variable is binary or categorical (logistic regression is more appropriate in such cases).

## Function details

We use the execute_glm_regression() function to fit either a linear or logistic regression model using statsmodels.

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

def execute_glm_regression(elr_dataframe_df, elr_outcome_str, elr_predictors_list,
                           model_type='linear', print_results=True, labels=False, reg_type="Multi"):
    """
    Executes a GLM (Generalized Linear Model) for linear or logistic regression.

    Parameters:
    - elr_dataframe_df: Pandas DataFrame containing the data.
    - elr_outcome_str: Name of the outcome variable.
    - elr_predictors_list: List of predictor variable names.
    - model_type: 'linear' for linear regression (Gaussian) or 'logistic' for logistic regression (Binomial).
    - print_results: If True, prints the results table.
    - labels: (Optional) Dictionary to map variable names to human-readable labels.
    - reg_type: Regression type ('uni' or 'multi') to rename the output columns.

    Returns:
    - summary_df: DataFrame with the model results.
    """

    # 1. Define the family based on model_type
    if model_type.lower() == 'logistic':
        family = sm.families.Binomial()
    elif model_type.lower() == 'linear':
        family = sm.families.Gaussian()
    else:
        raise ValueError("model_type must be 'linear' or 'logistic'")

    # 2. Build the formula string for the regression model
    formula = elr_outcome_str + ' ~ ' + ' + '.join(elr_predictors_list)

    # 3. Convert categorical variables to the 'category' type
    categorical_vars = elr_dataframe_df.select_dtypes(include=['object', 'category']).columns.intersection(elr_predictors_list)
    for var in categorical_vars:
        elr_dataframe_df[var] = elr_dataframe_df[var].astype('category')

    # 4. Fit the GLM model using the specified formula, data, and family
    model = smf.glm(formula=formula, data=elr_dataframe_df, family=family)
    result = model.fit()

    # 5. Extract the results table from the model summary
    summary_table = result.summary2().tables[1].copy()

    # 6. Process the results for logistic or linear regression
    if model_type.lower() == 'logistic':
        # For logistic regression, compute Odds Ratios and their confidence intervals
        summary_table['Odds Ratio'] = np.exp(summary_table['Coef.'])
        summary_table['IC Low'] = np.exp(summary_table['[0.025'])
        summary_table['IC High'] = np.exp(summary_table['0.975]'])

        summary_df = summary_table[['Odds Ratio', 'IC Low', 'IC High', 'P>|z|']].reset_index()
        summary_df = summary_df.rename(columns={'index': 'Study',
                                                  'Odds Ratio': 'OddsRatio',
                                                  'IC Low': 'LowerCI',
                                                  'IC High': 'UpperCI',
                                                  'P>|z|': 'p-value'})
    else:
        # For linear regression, use the coefficients and confidence intervals directly
        summary_df = summary_table[['Coef.', '[0.025', '0.975]', 'P>|z|']].reset_index()
        summary_df = summary_df.rename(columns={'index': 'Study',
                                                  'Coef.': 'Coefficient',
                                                  '[0.025': 'LowerCI',
                                                  '0.975]': 'UpperCI',
                                                  'P>|z|': 'p-value'})

    # 7. Map variable names to human-readable labels if a labels dictionary is provided
    if labels:
        def parse_variable_name(var_name):
            if var_name == 'Intercept':
                return labels.get('Intercept', 'Intercept')
            elif '[' in var_name:
                base_var = var_name.split('[')[0]
                level = var_name.split('[')[1].split(']')[0]
                base_var_name = base_var.replace('C(', '').replace(')', '').strip()
                label = labels.get(base_var_name, base_var_name)
                return f'{label} ({level})'
            else:
                var_name_clean = var_name.replace('C(', '').replace(')', '').strip()
                return labels.get(var_name_clean, var_name_clean)
        summary_df['Study'] = summary_df['Study'].apply(parse_variable_name)

    # 8. Reorder the columns for clarity
    if model_type.lower() == 'logistic':
        summary_df = summary_df[['Study', 'OddsRatio', 'LowerCI', 'UpperCI', 'p-value']]
    else:
        summary_df = summary_df[['Study', 'Coefficient', 'LowerCI', 'UpperCI', 'p-value']]

    # 9. Remove the letter 'T.' from categorical variable names
    summary_df['Study'] = summary_df['Study'].str.replace('T.', '')

    # 10. Format numerical values (round coefficients and confidence intervals, format p-values)
    for col in summary_df.columns[1:-1]:
        summary_df[col] = summary_df[col].round(3)
    summary_df['p-value'] = summary_df['p-value'].apply(lambda x: f'{x:.4f}')

    # 11. Optionally remove the intercept row if not needed
    summary_df = summary_df[summary_df['Study'] != 'Intercept']

    # 12. Rename columns based on the regression type (univariate or multivariate)
    if reg_type.lower() == 'uni':
        if model_type.lower() == 'logistic':
            summary_df.rename(columns={
                'OddsRatio': 'OddsRatio (uni)',
                'LowerCI': 'LowerCI (uni)',
                'UpperCI': 'UpperCI (uni)',
                'p-value': 'p-value (uni)'
            }, inplace=True)
        else:
            summary_df.rename(columns={
                'Coefficient': 'Coefficient (uni)',
                'LowerCI': 'LowerCI (uni)',
                'UpperCI': 'UpperCI (uni)',
                'p-value': 'p-value (uni)'
            }, inplace=True)
    elif reg_type.lower() == 'multi':
        if model_type.lower() == 'logistic':
            summary_df.rename(columns={
                'OddsRatio': 'OddsRatio (multi)',
                'LowerCI': 'LowerCI (multi)',
                'UpperCI': 'UpperCI (multi)',
                'p-value': 'p-value (multi)'
            }, inplace=True)
        else:
            summary_df.rename(columns={
                'Coefficient': 'Coefficient (multi)',
                'LowerCI': 'LowerCI (multi)',
                'UpperCI': 'UpperCI (multi)',
                'p-value': 'p-value (multi)'
            }, inplace=True)

    # 13. Print the results if print_results is True
    if print_results:
        print(summary_df)

    # 14. Return the summary DataFrame with all the model results
    return summary_df

### Inputs

* elr_dataframe_df: Pandas DataFrame containing the data.

* elr_outcome_str: Name of the outcome (dependent) variable.

* elr_predictors_list: List of predictor (independent) variables.

* model_type: 'linear' (default) or 'logistic'.

* print_results: If True, prints the results table.

* labels: (Optional) Dictionary to replace variable names with readable labels.

* reg_type: 'uni' or 'multi' – controls the column labels in the output.

### Output

**A summary DataFrame with:**

* Variable names (mapped to readable labels if provided)

* Coefficients or Odds Ratios

* 95% Confidence Intervals

* p-values

* Renamed columns depending on regression type (uni/multi)





## Example

###Importing

In this example we are going to predict the Respiratory Rate using age, sex, hypertension, diabetes, Highest recorded temperature and Creatinine level as predictors variables

In [3]:
from google.colab import files

uploaded = files.upload()


df = pd.read_csv('df_map.csv')


Saving df_map.csv to df_map (1).csv



* We load the dataset (df_map.csv) and the variable dictionary (vertex_dictionary.csv) using pd.read_csv().

* The variable dictionary contains technical variable names (field_name) and their corresponding human-readable labels (field_label).

* We then create a mapping dictionary (var_name_map) using zip() and dict() to associate each technical name with its readable label.

* This mapping is later used to replace technical variable names in the regression output with more understandable labels for interpretation and presentation.

### Define response variable and predictors

In [4]:
outcome_variable = "vital_rr"
predictor_variables = ["demog_age", "demog_sex", "comor_hypertensi",
                        "comor_diabetes_yn", "vital_highesttem_c", "labs_creatinine_mgdl"]



* **outcome_variable** → response variable (vital_rr).

* **predictor_variables** → list of predictor variables

### Run the linear regression model

In [9]:
summary_df = execute_glm_regression(elr_dataframe_df=df,
                                    elr_outcome_str=outcome_variable,
                                    elr_predictors_list=predictor_variables,
                                    model_type="linear",
                                    print_results=True,
                                    labels= False,
                                    reg_type="Multi")

                     Study  Coefficient (multi)  LowerCI (multi)  \
1          demog_sex[Male]                0.022           -0.167   
2   comor_hypertensi[True]               -0.010           -0.202   
3  comor_diabetes_yn[True]                0.304           -0.005   
4                demog_age                0.003           -0.001   
5       vital_highesttem_c                0.010           -0.070   
6     labs_creatinine_mgdl                0.020           -0.633   

   UpperCI (multi) p-value (multi)  
1            0.212          0.8175  
2            0.182          0.9174  
3            0.613          0.0542  
4            0.007          0.1278  
5            0.089          0.8109  
6            0.673          0.9525  


**The execute_glm_regression() function is called with all required parameters:**
* **elr_dataframe_df** = df → The dataset.

* **elr_outcome_str** = outcome_variable → The dependent variable (vital_rr).

* **elr_predictors_list** = predictor_variables  → List of independent variables.

* **model_type** = "linear" → We specify **linear regression**.

* **print_results** = True → Displays the regression output.

* **labels** = True → For mapping variable names to labels.

* **reg_type** = "Multi" → Specifies **multivariate regression**.

## Interpreting results

### Analysis of the Results

* **Sex (Male) (demog_sex[Male])** → Coefficient close to 0 (0.022), wide confidence interval, and p-value 0.8175 (not significant).

* **Age (demog_age)** → Small coefficient (0.003), p-value 0.1278 (not significant).

* **Hypertension (comor_hypertensi)** → Negative coefficient (-0.010), p-value 0.9174 (not significant).

* **Diabetes (comor_diabetes_yn)** → Positive coefficient (0.304), confidence interval includes 0, but p-value 0.0542 (almost significant).

* **Highest Temperature (vital_highesttem_c)** → Coefficient 0.010, p-value 0.8109 (not significant).

* **Creatinine (labs_creatinine_mgdl)** → Coefficient 0.020, p-value 0.9525 (not significant).

In this case None of the variables show strong statistical evidence to predict the outcome variable, as all p-values are greater than 0.05.

## Analyzing results with Forest Plot

The Forest Plot is a graphical method used to analyze the results table of linear regression. Each point represents an estimated coefficient (β), along with its lower and upper confidence intervals. This allows for a clear visualization of the direction (positive or negative) and the precision of the associations between the independent variables and the outcome.

In [10]:
import pandas as pd
import plotly.graph_objs as go


def fig_forest_plot(
        df, dictionary=None,
        title='Forest Plot',
        labels=['Study', 'OddsRatio', 'LowerCI', 'UpperCI'],
        graph_id='forest-plot', graph_label='', graph_about='',
        only_display=False):

    # Ordering Values -> Descending Order
    df = df.sort_values(by=labels[1], ascending=True)

    # Error Handling
    if not set(labels).issubset(df.columns):
        print(df.columns)
        error_str = f'Dataframe must contain the following columns: {labels}'
        raise ValueError(error_str)

    # Prepare Data Traces
    traces = []

    # Add the point estimates as scatter plot points
    traces.append(
        go.Scatter(
            x=df[labels[1]],
            y=df[labels[0]],
            mode='markers',
            name='Odds Ratio',
            marker=dict(color='blue', size=10))
    )

    # Add the confidence intervals as lines
    for index, row in df.iterrows():
        traces.append(
            go.Scatter(
                x=[row[labels[2]], row[labels[3]]],
                y=[row[labels[0]], row[labels[0]]],
                mode='lines',
                showlegend=False,
                line=dict(color='blue', width=2))
        )

    # Define layout
    layout = go.Layout(
        title=title,
        xaxis=dict(title='Coefficient'),
        yaxis=dict(
            title='', automargin=True, tickmode='array',
            tickvals=df[labels[0]].tolist(), ticktext=df[labels[0]].tolist()),
        shapes=[
            dict(
                type='line', x0=1, y0=-0.5, x1=1, y1=len(df[labels[0]])-0.5,
                line=dict(color='red', width=2)
            )],  # Line of no effect
        margin=dict(l=100, r=100, t=100, b=50),
        height=600
    )

    return go.Figure(data=traces, layout=layout)

### Executing Forest Plot

In [11]:
graph = fig_forest_plot(
    df = summary_df,
    labels = summary_df.columns.tolist(),
    only_display=True
)

graph.show()

# Model Valiation

In [ ]:



X = df[predictor_variables]


X = pd.get_dummies(X, drop_first=True)


X = sm.add_constant(X)

y = df[outcome_variable]




y = y.astype(float)
X = X.astype(float)


data = pd.concat([y, X], axis=1)
data_clean = data.dropna()

y_clean = data_clean[y.name]
X_clean = data_clean.drop(y.name, axis=1)



model = sm.OLS(y_clean, X_clean).fit()






## Assumptions

Are the theoretical conditions that should be met for the regression model to be valid and reliable:

<br><br>

### **Linearity:**


**What it means:**
The relationship between the independent variables and the dependent variable is linear.


**How to check:**
 Plot residuals vs. fitted values. A random scatter (no pattern) suggests linearity is satisfied. A curve or structure may indicate a non-linear relationship.



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=model.fittedvalues,
    y=model.resid,
    mode='markers',
    marker=dict(color='blue', size=8)
))

fig.add_hline(y=0, line_dash='dash', line_color='red')

fig.update_layout(
    title='Residuals vs Adjusted Values',
    xaxis_title='Adjusted Values',
    yaxis_title='Residuals',
    yaxis_range=[min(model.resid)*1.1, max(model.resid)*1.1]
)

fig.show()

**Interpretation: Linearity Assumption
From the plot:**

* The residuals are mostly scattered randomly around the red dashed horizontal line at 0.

* There is no clear curved or funnel-shaped pattern, which would have suggested a violation of linearity.

* The dispersion appears fairly uniform across the range of fitted values.

**Conclusion:**

The linearity assumption appears to be reasonably satisfied.
The relationship between the predictors and the outcome can be considered approximately linear based on this residual pattern.

<br><br>

### **Independence of Errors:**


**What it means:**
The residuals (errors) are independent of each other. No autocorrelation.


**How to check:**
 Use the Durbin-Watson test (for time series or ordered data).

The **Durbin-Watson statistic** is calculated as:

$$
D = \frac{\sum_{t=2}^{n} (e_t - e_{t-1})^2}{\sum_{t=1}^{n} e_t^2}
$$

Where:  
- \( et \) = residual (error) of observation \( t \)

- The **numerator** measures the variation between consecutive residuals.  

- The **denominator** measures the total variation in the residuals.


Interpretation:
- If the errors are **independent**, the variation between \( et \) and \( et-1 \) will be **large**, leading to:

  $$
  D \approx 2
  $$

- If the errors are **autocorrelated**, this variation will be **small**, and:

  $$
  D \text{ close to } 0 \text{ or } 4
  $$

<br><br>



In [ ]:
from statsmodels.stats.stattools import durbin_watson
dw = durbin_watson(model.resid)
print(f'Durbin-Watson Statistic: {dw:.3f}')

if dw < 1.5:
    print("→ Indicates positive autocorrelation of residuals.")
elif dw > 2.5:
    print("→ Indicates negative autocorrelation of residuals.")
else:
    print("→ Residuals are likely independent.")

Durbin-Watson Statistic: 1.977
→ Residuals are likely independent.


### **Homoscedasticity:**

**What it means:** The residuals have constant variance across all levels of the independent variables.


**How to check:**
Plot residuals vs. fitted values. A “funnel” shape (residuals increasing or decreasing with fitted values) indicates heteroscedasticity. You can also use statistical tests like Breusch–Pagan or White’s test.

<br><br>



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=model.fittedvalues,
    y=model.resid,
    mode='markers',
    marker=dict(color='blue', size=8)
))

fig.add_hline(y=0, line_dash='dash', line_color='red')

fig.update_layout(
    title='Residuals vs Adjusted Values',
    xaxis_title='Adjusted Values',
    yaxis_title='Residuals',
    yaxis_range=[min(model.resid)*1.1, max(model.resid)*1.1]
)

fig.show()


**Interpretation:**

The residuals (blue dots) are scattered randomly around the red dashed line (which represents zero).

There is no clear pattern, funnel shape, or systematic spread increasing or decreasing with the fitted values.

**Conclusion:**

The residuals appear to be randomly distributed around zero with roughly constant spread.
This suggests that the assumption of homoscedasticity is reasonably satisfied.

### **Normality of Errors:**

**What it means:** The residuals are approximately normally distributed (important for inference).

**How to check:** Create a Q-Q plot (quantile-quantile) of the residuals. Perform the Shapiro–Wilk test or Kolmogorov–Smirnov test. Histograms of residuals can also help.

In [ ]:
import scipy.stats as stats
import seaborn as sns


fig2 = go.Figure()
qq = stats.probplot(model.resid, dist="norm")
theoretical_quantiles = qq[0][0]
ordered_residuals = qq[0][1]

fig2.add_trace(go.Scatter(
    x=theoretical_quantiles,
    y=ordered_residuals,
    mode='markers',
    marker=dict(color='blue', size=8),
    name='Sample Quantiles'
))


fig2.add_trace(go.Scatter(
    x=theoretical_quantiles,
    y=theoretical_quantiles,
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Ideal Normal'
))

fig2.update_layout(
    title='Normality of Errors: Q-Q Plot',
    xaxis_title='Theoretical Quantiles',
    yaxis_title='Sample Quantiles',
)

fig2.show()


**Interpretation:**

* The blue dots represent the sample residuals.

* The red dashed line represents the expected values if the residuals were perfectly normally distributed.

* Most of the points lie close to the red dashed line, especially in the center.

* Some deviation is noticeable at the extremes (both tails), where points deviate from the straight line.

**Conclusion:**

The residuals are approximately normally distributed, especially in the middle of the distribution.

The slight deviation in the tails may indicate mild non-normality (e.g., heavier tails), but overall the assumption is reasonably satisfied for practical purposes.

In [ ]:
shapiro_test = stats.shapiro(model.resid)
print(f"Shapiro–Wilk test statistic: {shapiro_test.statistic:.4f}")
print(f"Shapiro–Wilk p-value: {shapiro_test.pvalue:.4f}")


if shapiro_test.pvalue > 0.05:
    print("Residuals appear to be normally distributed (fail to reject H0).")
else:
    print("Residuals do not appear to be normally distributed (reject H0).")

Shapiro–Wilk test statistic: 0.9969
Shapiro–Wilk p-value: 0.0461
Residuals do not appear to be normally distributed (reject H0).


**Conclusion:**

Since the p-value < 0.05, the assumption of normality is not fully satisfied.
However, the QQ plot showed only mild deviations in the tails, suggesting that this non-normality may not strongly affect model reliability, especially in large samples (due to the Central Limit Theorem).

### **No (Perfect) Multicollinearity:**

**What it means:** Independent variables are not perfectly correlated with each other.

**How to check:** Calculate the Variance Inflation Factor (VIF) for each predictor. Examine correlation matrices to identify highly correlated predictors.

The Variance Inflation Factor is calculated as:

VIF<sub>j</sub> = 1 / (1 - R²VIF<sub>j</sub>)

Where:

R²VIF<sub>j</sub> is the coefficient of determination obtained by regressing predictor Xj on all the other predictors in the model.

A higher VIF means stronger multicollinearity.

Interpretation:

**VIF = 1** No multicollinearity. The predictor is not correlated with other predictors.

**VIF between 1 and 5** Moderate multicollinearity. Usually acceptable, but monitor closely.

**VIF > 5** Potential multicollinearity problem. Consider investigating the correlated predictors.

**VIF ≥ 10** Severe multicollinearity. Strongly consider removing or combining variables.



In [ ]:
  from statsmodels.stats.outliers_influence import variance_inflation_factor


vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data = vif_data[vif_data["Variable"] != "const"]
print(vif_data)


               Variable       VIF
1             demog_age  1.029123
2      comor_hypertensi  1.024761
3     comor_diabetes_yn  2.085333
4    vital_highesttem_c  1.014304
5  labs_creatinine_mgdl  2.451858
6        demog_sex_Male  1.311005


**Interpretation:**

All variables have VIF < 5, indicating that multicollinearity is not a concern in the model.

There's no need to remove or transform any variables based on VIF at this point.

### **No Influential Outliers:**

**Note:** Influential outliers are not a formal mathematical assumption of linear regression. However, if ignored, they can significantly distort the model by pulling estimated coefficients away from the true trend. Observations with high leverage and large residuals may lead to misleading interpretations. Therefore, checking for influential points is strongly recommended in practice.

**What it means:** No single observation unduly influences the regression model.

**How to check:** Use Cook’s distance to detect influential points. Leverage plots and studentized residuals can also highlight outliers. Cook’s distance measures the influence of each data point on the overall regression results by assessing how much the fitted values would change if the point were removed. Values close to zero indicate little influence, while larger values suggest that the observation has a substantial impact on the estimated coefficients. A common rule of thumb is that points with Cook’s distance greater than 4 divided by the number of observations (4/n) may be considered influential. Points exceeding this threshold should be examined carefully to determine if they represent data errors, unusual but valid cases, or if they warrant alternative modeling approaches.

In [ ]:

influence = model.get_influence()


cooks_d = influence.cooks_distance[0]


obs = range(len(cooks_d))


threshold = 4 / len(cooks_d)


influential_points = [i for i, val in enumerate(cooks_d) if val > threshold]
print(f"Above limit points ({threshold:.3f}): {influential_points}")


Above limit points (0.004): [45, 90, 107, 117, 133, 154, 263, 274, 293, 310, 315, 347, 364, 437, 445, 462, 506, 540, 550, 551, 552, 580, 595, 614, 640, 661, 667, 695, 704, 705, 778, 785, 802, 926, 942, 944, 952, 956, 964]


**Interpretation:**

The analysis identified 39 observations with Cook's distance values above the threshold of 0.004, suggesting that they may have a notable influence on the model’s parameters. These points should be further investigated, but they are not necessarily errors or outliers that must be removed.

If these influential points correspond to valid data and are representative of important patterns or subgroups, they should be retained. However, if they are due to data entry errors or unrepresentative outliers, consider re-running the model after removing or adjusting them and comparing the results.

## Evaluating Predictive Quality

These are practical metrics used to assess whether the regression model can make accurate predictions on new, unseen data. This step is essential even if the model assumptions are satisfied, because a theoretically sound model may still perform poorly in practice.


####  Mean Squared Error (MSE)

**What it means:**  
The average of the squared differences between actual and predicted values. It gives more weight to large errors.

**How to check:**

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

Lower MSE indicates better model performance.




In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_clean, model.fittedvalues)
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 1.7592973987253377


**Interpretation:**

An MSE of 1.76 indicates that, on average, the squared prediction errors for the respiratory rate (vital_rr) are about 1.76 units². While MSE is useful for comparing models, its absolute meaning depends on the scale of the target variable. Since respiratory rate is typically measured in a small numeric range (e.g., 12–30 breaths per minute in clinical settings), an MSE near 1.76 may suggest a moderate predictive error. However, further comparison with simpler or more complex models, and evaluation through RMSE or MAE, can give a better idea of how well this model is performing.

<br><br>

####  Root Mean Squared Error (RMSE)

**What it means:**  
The square root of the MSE. Easier to interpret because it uses the same units as the response variable.

**How to check:**

$$
\text{RMSE} = \sqrt{\text{MSE}}
$$

Compare RMSE across models to assess predictive accuracy.



In [ ]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 1.3263850868904317


**Interpretation:**

An RMSE of approximately 1.33 means that, on average, the predicted values deviate from the true observed values by about 1.33 units. Since RMSE shares the same units as the dependent variable, this represents the typical size of the prediction errors made by the model. Whether this performance is considered good depends on the range and variability of the response variable. It is recommended to compare this RMSE with the scale of the data or against other models to better judge the model’s predictive quality.

<br><br>

####  Mean Absolute Error (MAE)

**What it means:**  
The average of the absolute differences between actual and predicted values. Less sensitive to outliers than MSE or RMSE.

**How to check:**

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

Lower MAE indicates better prediction accuracy.



In [ ]:
from sklearn.metrics import mean_absolute_error


mae = mean_absolute_error(y_clean, model.fittedvalues)

print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 1.0568156598119083


**Interpretation:**

An MAE of approximately 1.06 means that, on average, the model’s predictions differ from the true values by about 1.06 units. Because MAE is expressed in the same units as the response variable, this gives a direct sense of the average prediction error magnitude. Whether this value reflects good model performance depends on the scale and variability of the dependent variable. It is recommended to compare the MAE against the data scale or with other models to better assess predictive quality.

<br><br>

####  \( $R^2 $\) (Coefficient of Determination)

**What it means:**  
Represents the proportion of variance in the dependent variable explained by the model.

**How to check:**

$$
R^2 = 1 - \frac{SS_{res}}{SS_{tot}}
$$

Values closer to 1 indicate a better fit.  
However, it is not ideal for comparing models with different numbers of predictors.



In [ ]:
from sklearn.metrics import r2_score

# Assuming y_clean are the true values and model.fittedvalues are the predicted values
r2 = r2_score(y_clean, model.fittedvalues)

print("R² (Coefficient of Determination):", r2)


R² (Coefficient of Determination): 0.010197774892649725


<br><br>

####  Adjusted \( $R^2$ \)

**What it means:**  
A modified version of \($ R^2 $\) that adjusts for the number of predictors, penalizing the addition of irrelevant variables.

**How to check:**  
Compare adjusted \( $R^2$ \) when adding variables to check if the model actually improves.



In [ ]:
n = X_clean.shape[0]
p = X_clean.shape[1]

adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
print("Adjusted R²:", adjusted_r2)

Adjusted R²: 0.00321328338483573


<br><br>

####  Use of Test Data or Cross-Validation
**Note:** Not a model assumption, but a crucial part of performance evaluation.

**What it means:**  
It assesses how well the model generalizes to unseen data.
Always evaluate model performance on data that was not used during training to assess generalization.

**How to check:**  
- Split your dataset into **training** and **test** sets, or  
- Use **k-fold cross-validation** to get a more reliable performance estimate.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
# Define the model
model_cv = LinearRegression()

# Define the cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric (negative MSE because scikit-learn uses "higher is better" by default)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform cross-validation
cv_mse_scores = cross_val_score(model_cv, X_clean, y_clean, cv=kf, scoring=mse_scorer)

# Convert scores to positive MSE
cv_mse_scores = -cv_mse_scores

# Print results
print("Cross-Validation Mean Squared Errors (MSE):", cv_mse_scores)
print("Mean CV MSE:", np.mean(cv_mse_scores))
print("Standard Deviation of CV MSE:", np.std(cv_mse_scores))

Cross-Validation Mean Squared Errors (MSE): [1.80923187 1.59576084 1.79498889 2.1256914  1.59242271]
Mean CV MSE: 1.7836191401931085
Standard Deviation of CV MSE: 0.19475396038048962


**Interpretation:**

The model’s average Mean Squared Error across 5 folds is approximately 1.78, with a standard deviation of about 0.19. This indicates that, on average, the model’s predictions deviate from the actual values by a squared error of around 1.78 units. The relatively low standard deviation suggests that the model's performance is consistent across different subsets of the data, which supports the model’s ability to generalize to new, unseen data. However, this performance should still be judged in the context of the scale and variance of the target variable.

## References



* To learn more about numpy library, please go to the <a name='id_1'> <a href='https://numpy.org/'>Numpy WebPage</a>
<br>

* To learn more about pandas library, please go to the <a name='id_2'> <a href='https://pandas.pydata.org/'>Pandas WebPage</a>
<br>

* To learn more about statsmodels library, please go to the<a name='id_3'> <a href='https://www.statsmodels.org/stable/index.html'>StatsModels WebPage</a>
<br>

* To learn more about the Pip command, please go to the <a name='pip'>
<a href='https://pypi.org/project/pip/'>Pip WebPage</a>
<br>
